### Tutorial de  Julia para Otimização
## Escola de Verão - EMap/FGV
## Aula 04 - NLPModels e Comparação de Algortimos

### Ministrante
- Luiz-Rafael Santos ([LABMAC/UFSC/Blumenau](http://labmac.mat.blumenau.ufsc.br))
    * Email para contato: [l.r.santos@ufsc.br](mailto:l.r.santos@ufsc.br) ou [lrsantos11@gmail.com](mailto:lrsantos11@ufsc.br)
	- Repositório do curso no [Github](https://github.com/lrsantos11/Tutorial-Julia-Opt)


### Pacotes para contrução e comparação de algoritmos




* [JuliaSmoothOptimizers (JSO)](https://juliasmoothoptimizers.github.io/) coleção de pacotes em Julia para desenvolvimento, teste e *benchmark* de algoritmos de otimização (não-linear).
    
    * Modelagem
        * [NLPModels](https://github.com/JuliaSmoothOptimizers/NLPModels.jl): API para representar problemas de otimização `min f(x) s.t. l <= c(x) <= u`
    * Benchmark
        * [BenchmarkProfiles](https://github.com/JuliaSmoothOptimizers/BenchmarkProfiles.jl)
    * Respositórios de problemas
        * [CUTEst.jl](https://github.com/JuliaSmoothOptimizers/CUTEst.jl): interface para o [CUTEst](http://ccpforge.cse.rl.ac.uk/gf/project/cutest/wiki), repositório de problemas de otimização para teste  comparação de algoritmos de otimização.

In [ ]:
using Pkg
pkg"activate ../."
pkg"instantiate"

## Instalando os pacotes

* Vamos instalaros pacotes:
    * `NLPModels` para modelagem
    * `NLPModelsIpopt` `NLPModelsJuMP`que fazem interface entre NLPModels a Ipopt e JuMP
    * `CUTEst` para permitir usar os problemas da biblioteca [CUTEst](https://github.com/ralna/CUTEst/wiki)
	* `BenchmarkProfiles` para comparação de algoritmos

In [ ]:
pkg"add NLPModels NLPModelsIpopt NLPModelsJuMP CUTEst BenchmarkProfiles"

In [ ]:
using Plots, LinearAlgebra

## NLPModels


### Interfaces do NLPModels
#### [Interface Internas](https://juliasmoothoptimizers.github.io/NLPModels.jl/stable/#Internal-Interfaces)

 - `ADNLPModel`: Usa
   [`ForwardDiff`](https://github.com/JuliaDiff/ForwardDiff.jl) para computar derivadas. Simples mas não tão eficiente
   for larger problems.
 - `SlackModel`: Cria problemas com restrições de igualdade e restrições de caixa a partir de um NLPModel existente.
 - `LBFGSModel`: Cria modelo usando a aproximação LBFGS para a Hessiana a partir de um NLPModel existente.
 - `ADNLSModel`: Similar a  `ADNLPModel`, mas para qudarados mínimos não-lineares (nonlinear least squares)

#### [Interface Externas](https://juliasmoothoptimizers.github.io/NLPModels.jl/stable/#External-Interfaces)

 - `AmplModel`: Definida em 
   [`AmplNLReader.jl`](https://github.com/JuliaSmoothOptimizers/AmplNLReader.jl)
   para problemas modelados com [AMPL](https://ampl.com)
 - `CUTEstModel`: Definida em 
   [`CUTEst.jl`](https://github.com/JuliaSmoothOptimizers/CUTEst.jl) para problemas da biblioteca [CUTEst](https://github.com/ralna/CUTEst/wiki).
 - [`MathOptNLPModel`](https://github.com/JuliaSmoothOptimizers/NLPModelsJuMP.jl) e [`MathOptNLSModel`](https://github.com/JuliaSmoothOptimizers/NLPModelsJuMP.jl)
   para problemas modelados usando [JuMP.jl](https://github.com/jump-dev/JuMP.jl) e [MathOptInterface.jl](https://github.com/jump-dev/MathOptInterface.jl).

* NLPModels considera o problema de otimização dado da forma

$$\tag{P}
\begin{aligned}
\min \quad & f(x) \\
& c_L \leq c(x) \leq c_U \\
& \ell \leq x \leq u.
\end{aligned}
$$



### Exemplo - Função de Rosenbrock
Vamos novamente usar a função (não-linear) de Rosenbrock
$$f(x) = (1-x_1)^2 + 100(x_2-x_1^2)^2$$
para testar o pacote `NLPModels`.

In [ ]:
f(x) = (1-x[1])^2 + 100(x[2] - x[1]^2)^2

com ponto inicial $(-1.2,1.0)$.

In [ ]:
x0 = [-1.2,1.0]

* Vamos criar o modelo usando a interface `ADNLPModel`

O problema (P) em Julia será escrito como
```julia
 min  f(x)
s. a  lcon ≤ c(x) ≤ ucon
      lvar ≤   x  ≤ uvar
```
* As entradas de `ADNLPModel` podem ser

```julia
    ADNLPModel(f, x0)
    ADNLPModel(f, x0, lvar, uvar)
    ADNLPModel(f, x0, c, lcon, ucon)
    ADNLPModel(f, x0, lvar, uvar, c, lcon, ucon)
```

In [ ]:
using NLPModels
adnlp = ADNLPModel(f,x0)

### [API](https://juliasmoothoptimizers.github.io/NLPModels.jl/stable/api/#)



* Em otimização, como vimos, além dos valores $f(x)$ e $c(x)$ no ponto $x$ também precisamos das derivadas relacionadas, isto é,

- $\nabla f(x)$, $\nabla^2 f(x)$, $J_c(x) = \nabla c(x)^T$; 
- $\nabla^2 f(x) + \sum_{i=1}^m \lambda_i \nabla^2 c_i(x)$,
  Hessiana da função Lagrangeana no ponto $(x,\lambda)$.
> Para todas as funções disponibilizadas pelo pacote veja o [Guia de Referências](https://juliasmoothoptimizers.github.io/NLPModels.jl/stable/api/#Reference-guide) do API

* Vejam alguas que podemos usar

In [ ]:
@show obj(adnlp, adnlp.meta.x0)
@show grad(adnlp, adnlp.meta.x0)
@show hess(adnlp, adnlp.meta.x0)
@show objgrad(adnlp, adnlp.meta.x0)
@show hprod(adnlp, adnlp.meta.x0, ones(2))
@show H = hess_op(adnlp, adnlp.meta.x0)
H * ones(2)

In [ ]:
print(adnlp.counters)

#### Função `hess` em detalhes

* Apenas devolve o triangulo inferior da matriz Hessiana.



In [ ]:
Hx = hess(adnlp,adnlp.meta.x0)

In [ ]:
HxS = Symmetric(Hx,:L) #ou Hermitian(Hx,:L)

In [ ]:
F = cholesky(HxS)

### Usando modelos em JuMP

In [ ]:
using JuMP, NLPModelsJuMP
rosen = Model()

@variable(rosen,x[1:2])

@NLobjective(rosen,Min,(1-x[1])^2 + 100(x[2]-x[1]^2)^2)
print(rosen)

jpnlp = MathOptNLPModel(rosen)


### Usando a biblioteca CUTEst

* Um pouco mais sobre [CUTEst](https://github.com/ralna/CUTEst/wiki)
    > CUTEst, a Constrained and Unconstrained Testing Environment on steroids, is the latest evolution of CUTE, the constrained and unconstrained testing environment for numerical optimization.

* A interface com Julia que estamos usando permite acessar os problemas da CUTEst + outras bibliotecas (manualmente) que estão descritas no formato SIF.


In [ ]:
using CUTEst

cutenlp = CUTEstModel("ROSENBR")



In [ ]:
CUTEst.select()# Sempre preciso finalizar o modelo




In [ ]:
problmes2D = CUTEst.select(max_var=2,contype=:unc)

## Vamos usar nosso método de Newton


* Vamos voltar ao método de Newton que implementamos na Aula 02 porém 
    * o adaptaremos para uso do NLPModels
    * usaremos fatoração de Cholesky pra resolver o sistema linear de Newton
    * adicionaremos uma busca linear inexata com [Armijo com backtracking](https://en.wikipedia.org/wiki/Backtracking_line_search) para garantir descenso suficiente da direção de Newton $d_k$:
    
    

In [ ]:
function newton(nlp::AbstractNLPModel; itmax = 10_000,ε = 1e-6)
	k = 0
    xₖ = nlp.meta.x0
    gradₖ = grad(nlp,xₖ)
    while k <= itmax && norm(gradₖ) >= ε
        Hx = hess(nlp,xₖ)
        F = cholesky(Symmetric(Hx,:L))
        d = - (F \ gradₖ)
    	xₖ = xₖ + d 
        gradₖ = grad(nlp,xₖ)
    	k += 1
    end
    return xₖ, k
end

In [ ]:
@show xₖ, k = newton(adnlp)
print(adnlp.counters)

In [ ]:
jpnlp.meta.x0
@show xₖ, k = newton(jpnlp)
print(jpnlp.counters)


In [ ]:
cutenlp = CUTEstModel("ROSENBR")
@show xₖ, k = newton(cutenlp)
print(cutenlp.counters)



> **Armijo com backtracking.**
>   Enquanto  $f(x_k + \alpha_kd_k) > f(x_k) + \alpha_k\rho\nabla f(x_k)^Td_k $ escolha novo  $\alpha_k \in [0.1\alpha_k,0.9\alpha_k]$, para $\rho > 0$

In [ ]:

function newton_bt(nlp::AbstractNLPModel; itmax = 10_000,ε = 1e-6,ρ::Float64 = 0.5)
	k = 0
    xₖ = nlp.meta.x0
    fₖ = obj(nlp,xₖ)
    gradₖ = grad(nlp,xₖ)
    while k <= itmax && norm(gradₖ) >= ε
        Hx = hess(nlp,xₖ)
        F = cholesky(Symmetric(Hx,:L))
        dₖ = - (F \ gradₖ)
        αₖ = 1.0
        xtrial = xₖ + αₖ*dₖ 
        fα = obj(nlp,xtrial)
        while fα > fₖ + ρ * αₖ * dot(gradₖ,dₖ) 
            αₖ *= 0.5
            xtrial = xₖ + αₖ*dₖ 
            fα = obj(nlp,xtrial)
        end
        xₖ = xtrial
        fₖ = obj(nlp,xₖ)
        gradₖ = grad(nlp,xₖ)
    	k += 1
    end
    return xₖ, k
end

In [ ]:
newton_bt(cutenlp)

### Um exemplo que Newton BT não funciona

In [ ]:
nlp = CUTEstModel("HIMMELBB")

In [ ]:
newton_bt(nlp)

In [ ]:
finalize(nlp)

In [ ]:
nlp = CUTEstModel("BOX")

In [ ]:
newton_bt(nlp)

In [ ]:
newton(nlp)

## BenchmarkOrofiles
O pacote `BenchmarkProfiles` fornece uma fácil maneira de construir os Performance Profiles de Dolan e Moré. 

In [ ]:
using BenchmarkProfiles
gr()
T = 10 * rand(25,3);
plt = performance_profile(T, ["Solver 1", "Solver 2", "Solver 3"], title="Celebrity Deathmatch")
